<font face="Times New Roman" size=5>
<div dir=rtl align="center">
<font face="Times New Roman" size=5>
In The Name of God
</font>
<br>
<img src="https://logoyar.com/content/wp-content/uploads/2021/04/sharif-university-logo.png" alt="University Logo" width="150" height="150">
<br>
<font face="Times New Roman" size=4 align=center>
Sharif University of Technology - Department of Electrical Engineering
</font>
<br>
<font color="#008080" size=6>
Deep Learning
</font>
<hr/>
<font color="#800080" size=5>
Assignment 7 : Introduction to Graph Neural Networks
<br>
</font>
<font size=5>
Instructor: Dr. M. Bejani
<br>
</font>
<font size=4>
Spring 2025
<br>
</font>
<font face="Times New Roman" size=4>
Deadline: 10 Khordad 1404
</font>
<hr>
<font color='red'  size=4>
Note: It is highly recommended to run your notebook on Google Colab or Kaggle
<br>
</font>
<font face="Times New Roman" size=4 align=center>
Feel free to ask your questions in Telegram : @yasinsala
</font>
<br>
<hr>
</div></font>


#### 1. Pytorch Geometric Framework
- In this homework you get to familair with Pytorch Geometric (PyG) framework.
- Showcase the implementation of **Graph Convolution Networks** (Kipf & Welling, [SEMI-SUPERVISED CLASSIFICATION WITH GRAPH CONVOLUTIONAL NETWORKS](https://arxiv.org/abs/1609.02907), ICLR 2017), and you should implement **GraphSAGE** (Hamilton et al, [Inductive Representation Learning on Large Graphs](https://arxiv.org/abs/1706.02216), NIPS 2017) (You do not need to read the paper, just follow the hints).

#### 2. Vertex Classification
- Showcase a model developed based on our GCN implementation to do vertex classification on Cora dataset.
- Develop a model with **your own** GraphSAGE (with mean/sum/max aggregation) implementation on the same dataset to get insights of difference.


In [1]:
Name = "Seyyed Amirmahdi Sadrzadeh"
Student_Id = "401102015"

here we need to install the library we need

In [2]:
%%capture
!pip install torch-geometric

In [3]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="torch_geometric.data.in_memory_dataset")

#### Graph data representations in PyG
Given a *sparse* **Graph** $\mathcal{G}=(\mathbf{X}, (\mathbf{I}, \mathbf{E}))$ with **node features** $\mathbf{X} \in \mathbb{R}^{|V| \times F}$, **edge indices $\mathbf{I} \in \{1, \cdots, N\}^{2 \times |\mathcal{E}|}$**, (optional) **edge features** $\mathbf{E} \in \mathbb{R}^{|\mathcal{E} \times D|}$, it is described by an instance of class `torch_geometric.data.Data`, which holds the corresponding attributes.

We show a simple example of an unweighted and directed graph with four nodes and three edges.

<p align="center"><img width="20%" src="https://github.com/MahdiS3171/Deep-Learning-SUT-Course-Spring-2025/blob/main/HW/HW7/figures/graph_data.png?raw=1"></p>

In [4]:
# An example of creating a graph with 4 nodes and 3 edges in PyTorch Geometric

import torch
from torch_geometric.data import Data

# Define edge_index in COO format [2, num_edges]
edge_index = torch.tensor([
    [1, 2, 3],  # source nodes (x2, x3, x4)
    [0, 0, 0]   # target node (x1)
], dtype=torch.long)

# Define node features (here we use dummy features, e.g., 1 features per node)
x = torch.tensor([
    [1],  # x1
    [1],  # x2
    [1],  # x3
    [1],  # x4
], dtype=torch.float)

# Create the graph data object
data = Data(x=x, edge_index=edge_index)
data

Data(x=[4, 1], edge_index=[2, 3])

#### Abstract Message Passing Scheme in PyG

PyTorch Geometric provides the `torch_geometric.nn.MessagePassing` base class, which helps in creating such kinds of message passing graph neural networks by automatically taking care of message propagation. The implementation is decoupled into **UPDATE**, **AGGREGATION**, **MESSAGE** functions as:
$$
    \mathbf{x}_i^{(k)} = \mathrm{UPDATE} \left( \mathbf{x}_i, , \mathrm{AGGR}_{j \in \mathcal{N}(i)} \, \mathrm{MESSAGE}^{(k)}\left(\mathbf{x}_i^{(k-1)}, \mathbf{x}_j^{(k-1)},\mathbf{e}_{i,j}\right) \right)    
$$

<p align="center"><img width="35%" src="https://github.com/MahdiS3171/Deep-Learning-SUT-Course-Spring-2025/blob/main/HW/HW7/figures/message_passing.png?raw=1"></p>

#### Implementing the GCN layer (lecture)

The graph convolutional operator introduced by Kipf & Welling (ICLR 2017) is defined as
$$
        \mathbf{X}^{k} = \mathbf{\hat{D}}^{-1/2} \mathbf{\hat{A}}
        \mathbf{\hat{D}}^{-1/2} \mathbf{X}^{k-1} \mathbf{\Theta},
$$
where $\mathbf{\hat{A}} = \mathbf{A} + \mathbf{I}$ denotes the adjacency matrix with inserted self-loops and
$\hat{D}_{ii} = \sum_{j=0} \hat{A}_{ij}$ its diagonal degree matrix. It is equivalent as:
$$
\mathbf{x}_i^{(k)} = \sum_{j \in \mathcal{N}(i) \cup \{ i \}} \frac{1}{\sqrt{\deg(i)} \cdot \sqrt{deg(j)}} \cdot \left( \mathbf{x}_j^{(k-1)}\mathbf{\Theta} \right),
$$

where neighboring node features are first transformed by a weight matrix $\mathbf{\Theta}$, normalized by their degree, and finally summed up.
This formula can be divided into the following steps:

1. Add self-loops to the adjacency matrix.
2. Linearly transform node feature matrix.
3. Normalize node features.
4. Sum up neighboring node features.
5. Return new node embeddings.

In [5]:
import torch
from torch_geometric.nn import MessagePassing
import math

def glorot(tensor):
    if tensor is not None:
        stdv = math.sqrt(6.0 / (tensor.size(-2) + tensor.size(-1)))
        tensor.data.uniform_(-stdv, stdv)


def zeros(tensor):
    if tensor is not None:
        tensor.data.fill_(0)


def add_self_loops(edge_index, num_nodes=None):
    loop_index = torch.arange(0, num_nodes, dtype=torch.long,
                              device=edge_index.device)
    loop_index = loop_index.unsqueeze(0).repeat(2, 1)

    edge_index = torch.cat([edge_index, loop_index], dim=1)

    return edge_index


def degree(index, num_nodes=None, dtype=None):
    out = torch.zeros((num_nodes), dtype=dtype, device=index.device)
    return out.scatter_add_(0, index, out.new_ones((index.size(0))))


class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='add')  # "Add" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)

        self.reset_parameters()

    def reset_parameters(self):
        glorot(self.lin.weight)
        zeros(self.lin.bias)

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        ## TODO
        # Step 1: Add self-loops to the adjacency matrix.
        num_nodes = x.size(0)
        edge_index = add_self_loops(edge_index, num_nodes)

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)


        return self.propagate(edge_index, x=x, num_nodes=num_nodes)





    def message(self, x_j, edge_index, size):
        # x_j has shape [E, out_channels]

        ## TODO
        # Step 3: Normalize node features.
        # Calculate degree of source and target nodes
        row, col = edge_index
        deg = degree(row, size[0], dtype=x_j.dtype)  # degree of target nodes
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0

        # Normalize message
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        return norm.view(-1, 1) * x_j


    def update(self, aggr_out):
        # aggr_out has shape [N, out_channels]

        # Step 5: Return new node embeddings.
        return aggr_out

#### Implementing GraphSAGE (lab)

The algorithm of GraphSAGE (*Inductive Representation Learning on Large Graphs (NIPS 2017)*) embedding generation is described as:

<p align="center"><img width="45%" src="https://github.com/MahdiS3171/Deep-Learning-SUT-Course-Spring-2025/blob/main/HW/HW7/figures/graphsage.png?raw=1"></p>

You are required to implement this algortihm with **MEAN/SUM/MAX** AGGREGATE.

In [6]:
import torch
import torch.nn.functional as F
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
import math

def uniform(size, tensor):
    bound = 1.0 / math.sqrt(size)
    if tensor is not None:
        tensor.data.uniform_(-bound, bound)


class SAGEConv(MessagePassing):
    def __init__(self, in_channels, out_channels, aggr):
        super(SAGEConv, self).__init__(aggr=aggr)

        self.in_channels = in_channels
        self.out_channels = out_channels

        self.weight = Parameter(torch.Tensor(2 * in_channels, out_channels))

        self.reset_parameters()

    def reset_parameters(self):
        uniform(self.weight.size(0), self.weight)

    def forward(self, x, edge_index):

        ## TODO
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: propagate messages (aggregate neighbor features)
        aggr_out = self.propagate(edge_index, x=x)  # shape: [N, in_channels]

        # Step 2: concatenate node's own feature with aggregated neighbor features
        out = torch.cat([x, aggr_out], dim=1)  # shape: [N, 2*in_channels]

        # Step 3: apply linear transformation
        out = torch.matmul(out, self.weight)  # shape: [N, out_channels]

        # Step 4: apply non-linearity (usually ReLU) outside this layer, or in model

        return out



    def message(self, x_j, edge_weight):

        ## TODO
        # x_j has shape [E, in_channels], features of source nodes (neighbors)

        # For GraphSAGE, message is just the neighbor feature itself
        return x_j


    def update(self, aggr_out, x):

        ## TODO
        # aggr_out has shape [N, in_channels] after aggregation

        # Here we just return the aggregated neighbor features to be concatenated later
        return aggr_out



## Vertex Classification

In [7]:
import os
import os.path as osp
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T

path = osp.join(os.getcwd(), 'data', 'Cora')
dataset = Planetoid(path, 'Cora')

Processing...
Done!


In [8]:
import time

from torch import tensor
from torch.optim import Adam

# This function handles the training, evaluation, and timing of a GNN model over multiple runs.
def run(dataset, model, runs, epochs, lr, weight_decay, early_stopping):

    val_losses, accs, durations = [], [], []
    for _ in range(runs):
        data = dataset[0]
        data = data.to(device)


        model.to(device).reset_parameters()

        ## TODO
        # use Adam optimizer
        optimizer = Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

        if torch.cuda.is_available():
            torch.cuda.synchronize()

        t_start = time.perf_counter()

        best_val_loss = float('inf')
        test_acc = 0
        val_loss_history = []

        for epoch in range(1, epochs + 1):
            train(model, optimizer, data)
            eval_info = evaluate(model, data)
            eval_info['epoch'] = epoch

            if eval_info['val_loss'] < best_val_loss:
                best_val_loss = eval_info['val_loss']
                test_acc = eval_info['test_acc']

            val_loss_history.append(eval_info['val_loss'])

            if early_stopping > 0 and epoch > epochs // 2:
                tmp = tensor(val_loss_history[-(early_stopping + 1):-1])
                if eval_info['val_loss'] > tmp.mean().item():
                    break

        if torch.cuda.is_available():
            torch.cuda.synchronize()

        t_end = time.perf_counter()

        val_losses.append(best_val_loss)
        accs.append(test_acc)
        durations.append(t_end - t_start)

    loss, acc, duration = tensor(val_losses), tensor(accs), tensor(durations)

    print('Val Loss: {:.4f}, Test Accuracy: {:.3f} ± {:.3f}, Duration: {:.3f}'.
          format(loss.mean().item(),
                 acc.mean().item(),
                 acc.std().item(),
                 duration.mean().item()))


def train(model, optimizer, data):
    ## TODO
     # Step 1: Set model to training mode.
    model.train()
    # Step 2: Zero the gradients of the optimizer.
    optimizer.zero_grad()
    # Step 3: Forward pass through the model to get the output logits.
    out = model(data)
    # Step 4: Compute the loss using negative log likelihood loss.
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    # Step 5: Backward pass to compute gradients.
    loss.backward()
    # Step 6: Update the weights using the optimizer.
    optimizer.step()

def evaluate(model, data):
    model.eval()

    with torch.no_grad():
        logits = model(data)

    outs = {}
    for key in ['train', 'val', 'test']:
        mask = data['{}_mask'.format(key)]
        loss = F.nll_loss(logits[mask], data.y[mask]).item()
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()

        outs['{}_loss'.format(key)] = loss
        outs['{}_acc'.format(key)] = acc

    return outs

#### Build the model with GCN on vertex classification (lecture)

In [11]:
## TODO
# set hyperparameters
runs = 20
epochs = 200
lr = 0.01
weight_decay = 5e-4
early_stopping = 10
hidden = 16
dropout = 0.5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class Net(torch.nn.Module):
    def __init__(self, dataset):
        super(Net, self).__init__()

        ## TODO
        self.conv1 = GCNConv(dataset.num_node_features, hidden)
        self.conv2 = GCNConv(hidden, dataset.num_classes)

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()

    def forward(self, data):

        ## TODO
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


run(dataset, Net(dataset), runs, epochs, lr, weight_decay, early_stopping)

Val Loss: 0.7265, Test Accuracy: 0.804 ± 0.006, Duration: 0.557


#### Build models with GraphSAGE on vertex classification (lab)

In [12]:
## define your own model

class SAGENet(torch.nn.Module):
    def __init__(self, dataset, aggr='mean'):
        super(SAGENet, self).__init__()

        ## TODO
        self.conv1 = SAGEConv(dataset.num_node_features, hidden, aggr)
        self.conv2 = SAGEConv(hidden, dataset.num_classes, aggr)

        self.reset_parameters()


    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()

    def forward(self, data):

        ## TODO
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


aggrs = ['mean', 'add', 'max']

for aggr in aggrs:
    print('GraphSAGE-{}'.format(aggr))
    run(dataset, SAGENet(dataset, aggr), runs, epochs, lr, weight_decay,
        early_stopping)

GraphSAGE-mean
Val Loss: 0.7705, Test Accuracy: 0.794 ± 0.008, Duration: 0.686
GraphSAGE-add
Val Loss: 1.0731, Test Accuracy: 0.741 ± 0.032, Duration: 0.633
GraphSAGE-max
Val Loss: 0.9426, Test Accuracy: 0.761 ± 0.017, Duration: 0.743
